# PROGETTO DI FOCS 2020 

### ***Anna Nava, Marco Maugeri, Mario Pedol***
##### ***CdLM Data Science, Università degli Studi Milano-Bicocca.***

In [1]:
import pandas as pd
import numpy as np
import re 

In [2]:
times=pd.read_csv("timesData.csv")
cwur=pd.read_csv("cwurData.csv")
shanghai=pd.read_csv("shanghaiData.csv")
educational=pd.read_csv('educational_attainment_supplementary_data.csv')

In [3]:
#Si convertono i - in valori mancanti
times["income"]=times["income"].replace('-', np.NaN)

#Si trasforma income da stringa a float
times["income"]=times["income"].astype(float)



##### 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes. ##### 

In [4]:
#Si raggruppa per Università e si prende l'anno massimo 
timesRecent=times.iloc[times.groupby('university_name')['year'].idxmax()][['university_name','year','income','world_rank',
                                              'num_students','female_male_ratio','country']]


In [5]:
timesRecent.head() #Si stampano i risultati 

,university_name,year,income,world_rank,num_students,female_male_ratio,country
2405,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland
2003,Aalborg University,2016,43.7,201-250,"17,422",48 : 52,Denmark
2056,Aalto University,2016,61.6,251-300,"16,099",32 : 68,Finland
1908,Aarhus University,2016,68.3,=106,"23,895",54 : 46,Denmark
2105,Aberystwyth University,2016,31.3,301-350,"9,252",48 : 52,United Kingdom


In [6]:
#Si raggruppa per Università e si prende l'anno minimo 
timesOld=times.iloc[times.groupby('university_name')['year'].idxmin()][['university_name','year','income','world_rank',
                                              'num_students','female_male_ratio','country']]


In [7]:
timesOld.head() #Si stampano i risultati 

,university_name,year,income,world_rank,num_students,female_male_ratio,country
2405,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland
501,Aalborg University,2012,36.4,301-350,"17,422",48 : 52,Denmark
502,Aalto University,2012,61.9,301-350,"16,099",32 : 68,Finland
166,Aarhus University,2011,61.5,167,"23,895",54 : 46,Denmark
476,Aberystwyth University,2012,35.5,276-300,"9,252",48 : 52,United Kingdom


##### 2. For each university, compute the improvement in income between the least recent and the most recent data points.

In [8]:
#Si uniscono le tabelle precedentemente ottenute
improvement=pd.merge(timesRecent, timesOld, on=["university_name"], suffixes = ['_recent', '_old'])

#Si crea una nuova colonna "improvement" con il miglioramento del guadagno
improvement["improvement"]=(improvement["income_recent"]-improvement["income_old"])
improvement.head()

,university_name,year_recent,income_recent,world_rank_recent,num_students_recent,female_male_ratio_recent,country_recent,year_last,income_last,world_rank_last,num_students_last,female_male_ratio_last,country_last,improvement
0,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland,2016,NaN,601-800,"35,569",-,Poland,NaN
1,Aalborg University,2016,43.7,201-250,"17,422",48 : 52,Denmark,2012,36.4,301-350,"17,422",48 : 52,Denmark,7.3
2,Aalto University,2016,61.6,251-300,"16,099",32 : 68,Finland,2012,61.9,301-350,"16,099",32 : 68,Finland,-0.3
3,Aarhus University,2016,68.3,=106,"23,895",54 : 46,Denmark,2011,61.5,167,"23,895",54 : 46,Denmark,6.8
4,Aberystwyth University,2016,31.3,301-350,"9,252",48 : 52,United Kingdom,2012,35.5,276-300,"9,252",48 : 52,United Kingdom,-4.2


##### 3. Find the university with the largest increase computed in the previous point.

In [9]:
improvement[improvement['improvement']>=improvement['improvement'].max()]

,university_name,year_last,income_last,world_rank_last,num_students_last,female_male_ratio_last,country_last,year_recent,income_recent,world_rank_recent,num_students_recent,female_male_ratio_recent,country_recent,improvement
428,TU Dresden,2016,99.7,=158,"35,487",42 : 58,Germany,2012,31.9,251-275,"35,487",42 : 58,Germany,67.8


##### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range.

In [10]:
#Prendiamo i dati più recenti per tutti i rank


###############
shanghaiRank = shanghai.iloc[shanghai.groupby('university_name')['year'].idxmax()][['university_name','year','world_rank']]
shanghaiRank.rename(columns={'year':'year_shanghai','world_rank':'world_rank_shanghai'}, inplace=True)

###############
timesRank = timesRecent[['university_name','year','world_rank']]
timesRank.rename(columns={'year':'year_time','world_rank':'world_rank_time'}, inplace=True)

###############
cwurRank = cwur.iloc[cwur.groupby('institution')['year'].idxmax()][['institution','year','world_rank']]
cwurRank.rename(columns={'institution':'university_name','year':'year_cwur','world_rank':'world_rank_cwur'}, inplace=True)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
#Uniamo tutte le tabelle. how='outer' così da considerare tutti i casi

cwurTimes = pd.merge(cwurRank, timesRank, on='university_name', how='outer')
rank = pd.merge(cwurTimes, shanghaiRank, on='university_name', how='outer')


In [12]:
#Implementiamo una funzione per estrarre le differenze massime

def maxDiffRank(rank):
    #Suffisso per scorrere sulle colonne di rank
    namesRank = ['time','shanghai','cwur'] 
    diffRank=[]
    for name in namesRank:
        if pd.notna(rank['world_rank_'+name]): #Non si considerano i valori nulli
            if re.search('-',str(rank['world_rank_'+name])) is not None: #Se c'è un range si calcola la media
                estremi=rank['world_rank_'+name].split('-') #Separiamo gli estremi
                diffRank.append((int(estremi[0])+int(estremi[1]))/2) #Calcolo media
            else:
                #Prendiamo il valore numerico del rank
                diffRank.append(int(re.search('(\d+)',str(rank['world_rank_'+name])).group(0)))
    #Restituiamo la differenza massima            
    return max(diffRank)-min(diffRank)   


In [13]:
#Applichiamo la funzione appena creata
rank['maxDiffRank'] = rank.apply(maxDiffRank, axis=1)
rank.head()


,university_name,year_cwur,world_rank_cwur,year_time,world_rank_time,year_shanghai,world_rank_shanghai,maxDiffRank
0,AGH University of Science and Technology,2015.0,782.0,2016.0,601-800,NaN,NaN,81.5
1,Aalborg University,2015.0,565.0,2016.0,201-250,2015.0,301-400,339.5
2,Aalto University,2015.0,421.0,2016.0,251-300,2015.0,401-500,175.0
3,Aarhus University,2015.0,122.0,2016.0,=106,2015.0,73,49.0
4,Aberystwyth University,2015.0,814.0,2016.0,301-350,NaN,NaN,488.5


In [14]:
#Controllo
rank[rank['university_name']=='Aarhus University']

,university_name,year_cwur,world_rank_cwur,year_time,world_rank_time,year_shanghai,world_rank_shanghai,maxDiffRank
3,Aarhus University,2015.0,122.0,2016.0,=106,2015.0,73,49.0


##### 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [15]:
#Convertiamo i simboli "-" in valori mancanti
timesRecent["female_male_ratio"]=times["female_male_ratio"].replace('-', np.NaN)

#Si eliminano i valori nulli
timesRecentNonull = timesRecent.dropna(subset=['female_male_ratio','num_students'])

#Convertiamo stringa in intero
timesRecentNonull['num_students'] = timesRecentNonull['num_students'].str.replace(',', '').astype(int)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#Si creano due gruppi float con percentuali di femmine e maschi
genere = timesRecentNonull['female_male_ratio'].str.extract(r'(?P<female_percentage>^\d+) : (?P<male_percentage>\d+$)').astype(float)

#Si crea una colonna con il numero di femmine
timesRecentNonull['female']=((genere.female_percentage/100)*timesRecentNonull['num_students']).round(0)

#Si crea una colonna con il numero di maschi
timesRecentNonull['male']=((genere.male_percentage/100)*timesRecentNonull['num_students']).round(0)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
timesRecentNonull[['university_name','male','female']]

,university_name,male,female
2003,Aalborg University,9059.0,8363.0
2056,Aalto University,10947.0,5152.0
1908,Aarhus University,10992.0,12903.0
2105,Aberystwyth University,4811.0,4441.0
2404,Adam Mickiewicz University,11784.0,28849.0
...,...,...,...
1856,École Normale Supérieure,1296.0,1104.0
2013,École Normale Supérieure de Lyon,1131.0,1087.0
1904,École Polytechnique,1992.0,437.0
1833,École Polytechnique Fédérale de Lausanne,7056.0,2610.0


##### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [34]:
#Creiamo due colonne aggiuntive con la percentuali di maschi e di femnmine
timesRecentNonull['female_percentage']=genere.female_percentage
timesRecentNonull['male_percentage']=genere.male_percentage

#Si elimina dal dataset la tupla contente il valore percentuale di male pari a 0
#perchè non è possibile calcolare il rapporto al punto che segue
DataRatio=timesRecentNonull.drop(timesRecentNonull[timesRecentNonull['male_percentage']==0].index)

#Si calcola il rapporto tra female e male
DataRatio['ratio_female_male']=DataRatio['female_percentage']/DataRatio['male_percentage']

#Si selezionano tutte le università che hanno il rapporto inferiore alla media
low_female_university = DataRatio[DataRatio['ratio_female_male'] < DataRatio['ratio_female_male'].mean()]
low_female_university['university_name']


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2003                          Aalborg University
2056                            Aalto University
2105                      Aberystwyth University
2406                             Ajou University
2408                       Alexandria University
                          ...                   
2104                         Zhejiang University
1856                    École Normale Supérieure
2013            École Normale Supérieure de Lyon
1904                         École Polytechnique
1833    École Polytechnique Fédérale de Lausanne
Name: university_name, Length: 346, dtype: object

##### 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [43]:
#Calcoliamo il totale degli studenti per ogni nazione
students_number_tot = timesRecentNonull.groupby('country')['num_students'].sum()

#Calcoliamo il totale degli studenti per ogni nazione apparteneti alle università calcolate al punto precedente
students_number =low_female_university.groupby('country')['num_students'].sum()

#Si stampano i risultati
(students_number/students_number_tot).dropna().head(5)


country
Australia     0.204990
Austria       0.453855
Bangladesh    1.000000
Brazil        0.564033
Canada        0.139152
Name: num_students, dtype: float64

##### 8. Read the file educational_attainment_supplementary_data.csv, discarding any row without country_name or series_name

In [49]:
educational.dropna(axis=0, subset=['country_name','series_name'], inplace=True)
educational.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


##### 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [50]:
#Si utilizza il metodo melt per fare unpivot del dataset, ossia si passa da un formato wide a uno long
educationalLong=pd.melt(educational, id_vars=['country_name', 'series_name'])
educationalLong.rename(columns={'variable':'year'}, inplace=True)
educationalLong

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


##### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [53]:
#Aggiungiamo ai rank una colonna 1 che segna la presenza dell'università nel dataset
#eliminando i duplicati si avrà un dataset dove ad ogni uni è associato il valore 1
times['times']=1
universityTimes= times[['university_name','times']].drop_duplicates()

shanghai['shanghai']=1
universityShanghai= shanghai[['university_name','shanghai']].drop_duplicates()

cwur['cwur']=1
universityCwur= cwur[['institution', 'cwur']].drop_duplicates()
universityCwur.rename(columns={'institution': 'university_name'}, inplace=True)

#Si uniscono le tabelle tenendo tutti i record e si associa il valore 0 se un università è mancante in uno dei rank 
t_s =pd.merge(how='outer', left= universityTimes, right=universityShanghai, on='university_name' )
number_university =pd.merge(how='outer', left= t_s, right=universityCwur, on='university_name' )
number_university.fillna(value=0, inplace=True)

#Si sommano le università che sono presenti in almeno un rank
number_university['sum'] = number_university['times']+number_university['shanghai']+number_university['cwur']
number_university

,university_name,times,shanghai,cwur,sum
0,Harvard University,1.0,1.0,1.0,3.0
1,California Institute of Technology,1.0,1.0,1.0,3.0
2,Massachusetts Institute of Technology,1.0,0.0,1.0,2.0
3,Stanford University,1.0,1.0,1.0,3.0
4,Princeton University,1.0,1.0,1.0,3.0
...,...,...,...,...,...
1448,Shenzhen University,0.0,0.0,1.0,1.0
1449,Tianjin Medical University,0.0,0.0,1.0,1.0
1450,Babeș-Bolyai University,0.0,0.0,1.0,1.0
1451,Henan Normal University,0.0,0.0,1.0,1.0


##### 11. In the times ranking, compute the number of times each university appears.

In [60]:
uniTimeRank = times.groupby('university_name')['world_rank'].count()
uniTimeRank

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
                                           ..
École Normale Supérieure                    6
École Normale Supérieure de Lyon            6
École Polytechnique                         6
École Polytechnique Fédérale de Lausanne    6
Örebro University                           1
Name: world_rank, Length: 818, dtype: int64

##### 12. Find the universities that appear at most twice in the times ranking.

In [61]:
uniTimeRank[uniTimeRank <= 2]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Name: world_rank, Length: 408, dtype: int64

##### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [113]:
#Si crea una funzione 
def rank_good(data):
    #Si estraggono i due estremi del rank e se ne calcola la media
    estremi = data.str.extract(r'(?P<sinistro>^\d+)-(?P<destro>\d+$)').fillna(value=0).astype(float)
    estremi = (estremi['sinistro'] + estremi['destro'])/2
    #Si estrae i rank che presentano il simbolo "=" all'inizio della cella
    ugualeInizio = data.str.extract(r'^=*(\d+$)').fillna(value=0).astype(float)
    return estremi + ugualeInizio[0]

In [114]:
#Si rinominano i nomi delle variabili e si applica la funzione appena creata
cwur.rename(columns={'institution' : 'university_name', 'world_rank' : 'world_rank_good'}, inplace=True)
times['world_rank_good'] = rank_good(times['world_rank'])
shanghai['world_rank_good'] = rank_good(shanghai['world_rank'])

In [115]:
#Si uniscono tutti i dataset, prendendo solo le università con il rank uguale
uguali_ts = pd.merge(right = times[['university_name', 'world_rank_good', 'year']], 
                     left = shanghai[['university_name', 'world_rank_good', 'year']],
                    on=['university_name',  'year', 'world_rank_good'])
uguali_tsc = pd.merge(right = uguali_ts[['university_name', 'world_rank_good', 'year']], 
                     left = cwur[['university_name', 'world_rank_good', 'year']],
                    on=['university_name',  'year', 'world_rank_good'])
uguali_tsc

,university_name,world_rank_good,year
0,Stanford University,2,2013
